In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1024, 1024)
        self.lin1 = torch.nn.Linear(1024, 1024)
        self.conv2 = GCNConv(1024, 1024)
        self.lin2 = torch.nn.Linear(1024, 1024)
#         self.conv3 = GCNConv(1024, 1024)
#         self.lin3 = torch.nn.Linear(1024, 1024)

        self.lin4 = torch.nn.Linear(2048, 512)
        self.lin5 = torch.nn.Linear(512, 256)
#         self.lin6 = torch.nn.Linear(128, 1)
#         self.lin2 = torch.nn.Linear(256, 128)

    def forward(self, pos_edge_index, neg_edge_index):

        x1 = F.elu(self.conv1(data.x, data.train_pos_edge_index)+ self.lin1(data.x))
        x1 = F.dropout(x1, p=0.5, training=self.training)
        x2 = F.elu(self.conv2(x1, data.train_pos_edge_index)+ self.lin2(data.x))
        x2 = F.dropout(x2, p=0.25, training=self.training)

#         x = F.elu(self.conv2(x, data.train_pos_edge_index)+ self.lin2(x)) 
#         x = F.elu(self.conv3(x, data.train_pos_edge_index)+ self.lin3(x))
        x = torch.cat([x1, x2], dim=-1)

        x = F.elu(self.lin4(x))
        x = F.dropout(x, p=0.5, training=self.training)

        x = F.elu(self.lin5(x))
#         x = F.elu(self.lin6(x))
        
        total_edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
        x_j = torch.index_select(x, 0, total_edge_index[0])
        x_i = torch.index_select(x, 0, total_edge_index[1])
        return torch.einsum("ef,ef->e", x_i, x_j)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model, data = Net().to(device), data.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.005)


def get_link_labels(pos_edge_index, neg_edge_index):
    link_labels = torch.zeros(pos_edge_index.size(1) +
                              neg_edge_index.size(1)).float().to(device)
    link_labels[:pos_edge_index.size(1)] = 1.
    return link_labels


def train():
    model.train()
    optimizer.zero_grad()

    x, pos_edge_index = data.x, data.train_pos_edge_index

    _edge_index, _ = remove_self_loops(pos_edge_index)
    pos_edge_index_with_self_loops, _ = add_self_loops(_edge_index,
                                                       num_nodes=x.size(0))

    neg_edge_index = negative_sampling(
        edge_index=pos_edge_index_with_self_loops, num_nodes=x.size(0),
        num_neg_samples=pos_edge_index.size(1))

    link_logits = model(pos_edge_index, neg_edge_index)
    link_labels = get_link_labels(pos_edge_index, neg_edge_index)

    loss = F.binary_cross_entropy_with_logits(link_logits, link_labels)
    loss.backward()
    optimizer.step()

    return loss


def test():
    model.eval()
    perfs = []
    for prefix in ["test"]:
        pos_edge_index, neg_edge_index = [
            index for _, index in data("{}_pos_edge_index".format(prefix),
                                       "{}_neg_edge_index".format(prefix))
        ]
        link_probs = torch.sigmoid(model(pos_edge_index, neg_edge_index))
        link_labels = get_link_labels(pos_edge_index, neg_edge_index)
        link_probs = link_probs.detach().cpu().numpy()
        link_labels = link_labels.detach().cpu().numpy()
        perfs.append(roc_auc_score(link_labels[idx_c1], link_probs[idx_c1]))
        perfs.append(roc_auc_score(link_labels[idx_c2], link_probs[idx_c2]))
        perfs.append(roc_auc_score(link_labels[idx_c3], link_probs[idx_c3]))
        print(sum(link_labels[idx_c1]),sum(link_labels[idx_c2]),sum(link_labels[idx_c3]))

    return perfs


best_val_perf = test_perf = 0
for epoch in range(1, 501):
    train_loss = train()
    test_c1, test_c2, test_c3 = test()
 
    log = 'Epoch: {:03d}, Loss: {:.4f}, Test C1: {:.4f}, Test C2: {:.4f}, Test C3: {:.4f}'
    print(log.format(epoch, train_loss, test_c1, test_c2, test_c3))

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1024, 1024)
        self.lin1 = torch.nn.Linear(1024, 1024)
        self.conv2 = GCNConv(1024, 1024)
        self.lin2 = torch.nn.Linear(1024, 1024)
#         self.conv3 = GCNConv(1024, 1024)
#         self.lin3 = torch.nn.Linear(1024, 1024)

        self.lin4 = torch.nn.Linear(2048, 512)
        self.lin5 = torch.nn.Linear(512, 256)
#         self.lin6 = torch.nn.Linear(128, 1)
#         self.lin2 = torch.nn.Linear(256, 128)

    def forward(self, pos_edge_index, neg_edge_index):

        x1 = F.elu(self.conv1(data.x, data.train_pos_edge_index)+ self.lin1(data.x))
        x1 = F.dropout(x1, p=0.65, training=self.training)
        x2 = F.elu(self.conv2(x1, data.train_pos_edge_index)+ self.lin2(data.x))
        x2 = F.dropout(x2, p=0.45, training=self.training)

#         x = F.elu(self.conv2(x, data.train_pos_edge_index)+ self.lin2(x)) 
#         x = F.elu(self.conv3(x, data.train_pos_edge_index)+ self.lin3(x))
        x = torch.cat([x1, x2], dim=-1)

        x = F.elu(self.lin4(x))
        x = F.dropout(x, p=0.5, training=self.training)

        x = F.elu(self.lin5(x))
#         x = F.elu(self.lin6(x))
        
        total_edge_index = torch.cat([pos_edge_index, neg_edge_index], dim=-1)
        x_j = torch.index_select(x, 0, total_edge_index[0])
        x_i = torch.index_select(x, 0, total_edge_index[1])
        return torch.einsum("ef,ef->e", x_i, x_j)
